In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
dfs = pd.read_csv('data/2019-06-26/S_seaice_extent_daily_v3.0.csv', skiprows=[1])
dfn = pd.read_csv('data/2019-06-26/N_seaice_extent_daily_v3.0.csv', skiprows=[1])

dfs.columns = ['Year', 'Month', 'Day', 'Extent', 'Missing', 'Source Data']
dfn.columns = ['Year', 'Month', 'Day', 'Extent', 'Missing', 'Source Data']

dfs['date'] = pd.to_datetime(dfs['Year']*10000+dfs['Month']*100+dfs['Day'],format='%Y%m%d')
dfn['date'] = pd.to_datetime(dfn['Year']*10000+dfn['Month']*100+dfn['Day'],format='%Y%m%d')

dfn.index=dfn['date']
dfn = dfn.resample('2D').mean()

dfs.index=dfs['date']
dfs = dfs.resample('2D').mean()

In [ ]:
plt.figure(figsize=(8,4), dpi=150)

plt.plot(dfn.index,dfn['Extent'].rolling(182).mean(), label='northern')

plt.fill_between(
    dfn.index,
    dfn['Extent'].rolling(182).mean()+dfn['Extent'].rolling(182).std()/np.sqrt(182),
    dfn['Extent'].rolling(182).mean()-dfn['Extent'].rolling(182).std()/np.sqrt(182),
    alpha=0.5,
    label="uncert. 1$\sigma$"
    )

plt.plot(dfs.index,dfs['Extent'].rolling(182).mean(), label='southern')

plt.fill_between(
    dfs.index,
    dfs['Extent'].rolling(182).mean()+dfs['Extent'].rolling(182).std()/np.sqrt(182),
    dfs['Extent'].rolling(182).mean()-dfs['Extent'].rolling(182).std()/np.sqrt(182),
    alpha=0.5,
    label="uncert. 1$\sigma$"
    )

plt.legend()
plt.title('yearly rolling average')
plt.xlabel('date')
plt.ylabel('sea ice coverage / $10^6$ km$^2$')

In [ ]:
dtn = dfn
dts = dfs

plt.figure(figsize=(8,4), dpi=150)

plt.plot(dtn.index,dtn['Extent'].rolling(1).mean(), label='northern')
plt.plot(dts.index,dts['Extent'].rolling(1).mean(), label='southern')

plt.legend()
plt.title('full NSIDC dataset')
plt.xlabel('date')
plt.ylabel('sea ice coverage / $10^6$ km$^2$')

In [ ]:
dtn = dfn[dfn.index > pd.to_datetime('2012-12-31')]
dts = dfs[dfs.index > pd.to_datetime('2012-12-31')]

plt.figure(figsize=(8,4), dpi=150)

plt.plot(dtn.index,dtn['Extent'].rolling(1).mean(), label='northern')
plt.plot(dts.index,dts['Extent'].rolling(1).mean(), label='southern')

plt.legend()
plt.title('recent 5 years of NSIDC dataset')
plt.xlabel('date')
plt.ylabel('sea ice coverage / $10^6$ km$^2$')

In [ ]:
plt.figure(figsize=(8,4), dpi=150)

plt.plot(dtn.index,dtn['Extent'].rolling(182).mean(), label='northern')

plt.fill_between(
    dtn.index,
    dtn['Extent'].rolling(182).mean()+dtn['Extent'].rolling(182).std()/np.sqrt(182),
    dtn['Extent'].rolling(182).mean()-dtn['Extent'].rolling(182).std()/np.sqrt(182),
    alpha=0.5,
    label="uncert. 1$\sigma$"
    )

plt.plot(dts.index,dts['Extent'].rolling(182).mean(), label='southern')

plt.fill_between(
    dts.index,
    dts['Extent'].rolling(182).mean()+dts['Extent'].rolling(182).std()/np.sqrt(182),
    dts['Extent'].rolling(182).mean()-dts['Extent'].rolling(182).std()/np.sqrt(182),
    alpha=0.5,
    label="uncert. 1$\sigma$"
    )

plt.legend()
plt.title('recent 5 years, yearly average')
plt.xlabel('date')
plt.ylabel('sea ice coverage / $10^6$ km$^2$')

In [ ]:
plt.figure(figsize=(8,4), dpi=150)

for y in np.arange(2015,2020)[::-1]:
    dt = dfn[
        (dfn.index >= pd.to_datetime(str(y)+'-01-01')) & 
        (dfn.index <  pd.to_datetime(str(y)+'-12-31'))
    ]
    
    plt.plot(
        (dt.index-pd.to_datetime(str(y)+'-01-01'))/pd.Timedelta('1d'),
        dt['Extent'].rolling(1).mean(),
        label=y,
    )

plt.legend()

In [ ]:
plt.figure(figsize=(8,4), dpi=150)

for y in np.arange(2015,2020)[::-1]:
    dt = dfs[
        (dfs.index >= pd.to_datetime(str(y)+'-01-01')) & 
        (dfs.index <  pd.to_datetime(str(y)+'-12-31'))
    ]
    
    plt.plot(
        (dt.index-pd.to_datetime(str(y)+'-01-01'))/pd.Timedelta('1d'),
        dt['Extent'].rolling(1).mean(),
        label=y,
    )

plt.legend()

In [ ]:
np.arange(2020,2009)

In [ ]:
def plot_critical_years(df, c_start, c_end):

    plt.figure(figsize=(8,4), dpi=150)

    dtn = df[ 
            (df.index <  pd.to_datetime(str(c_start)+'-12-31'))
        ]

    daily_avg_val = dtn.resample('1D').interpolate().groupby(
        dtn.resample('1D').interpolate().index.dayofyear
    ).mean()['Extent']

    daily_avg_std = dtn.resample('1D').interpolate().groupby(
        dtn.resample('1D').interpolate().index.dayofyear
    ).std()['Extent']

    plt.plot(
        daily_avg_val,
        label='daily avg  until 2014'
    )

    plt.fill_between(
        np.arange(0,366),
        daily_avg_val+daily_avg_std,
        daily_avg_val-daily_avg_std,
        alpha=0.5
    )

    for y in np.arange(c_start,c_end):
        dt = df[
            (df.index >= pd.to_datetime(str(y)+'-01-01')) & 
            (df.index <  pd.to_datetime(str(y)+'-12-31'))
        ]

        plt.plot(
            (dt.index-pd.to_datetime(str(y)+'-01-01'))/pd.Timedelta('1d'),
            dt['Extent'].rolling(1).mean(),
            label=y,
        )

    plt.legend()
    plt.xlabel('days of year')
    plt.ylabel('sea ice coverage / $10^6$ km$^2$')

In [ ]:
plot_critical_years(dfn,2015,2020)

In [ ]:
plot_critical_years(dfs,2017,2020)

In [ ]:
df = dfn+dfs

In [ ]:
plot_critical_years(df,2015,2020)